In [1]:
%%capture
!pip install --upgrade scikit-learn fastdtw scipy

In [10]:
import os, glob
import numpy as np
import pandas as pd
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [4]:
# Funcion para cargar los datos
def load_bpm_series(file_path):
    with open(file_path, "r") as f:
        lines = f.readlines()
    bpm_values = []
    for line in lines:
        try:
            bpm = float(line.strip())
            bpm_values.append(bpm)
        except:
            continue
    return np.array(bpm_values)



# Cargar datos
txt_files = glob.glob(os.path.join("Datos", "*.txt"))
subject_series = {}  

for file_path in txt_files:
    subject = os.path.splitext(os.path.basename(file_path))[0].lower()
    series = load_bpm_series(file_path)
    subject_series[subject] = series

In [5]:
def convert_mmss_to_seconds(time_str):
    minutes, seconds = time_str.split(":")
    return int(minutes) * 60 + int(seconds)

obs_df = pd.read_csv("Datos/Heart_Rate_VR_Data.csv")
obs_df["subject"] = obs_df["subject"].str.lower()

obs_df["time"] = obs_df["time"].apply(convert_mmss_to_seconds)


Aqui segmento los datos con un empalmamiento de 50% entre ellos  para evitar que la parte que causa el miedo se encuentre por el final y no se capture completa.

In [6]:
def segment_series(series, window_size, step_size):
    segments = []
    for start in range(0, len(series) - window_size + 1, step_size):
        segments.append(series[start:start+window_size])
    return np.array(segments)
    
# Analizar ventanas de 10 segundos
window_size = 10   
# Avanzamos de 5 en 5 segundos
step_size = 5      

dataset_segments = []
dataset_labels = []

for subject, series in subject_series.items():
    segments = segment_series(series, window_size, step_size)
    times = np.arange(len(series))
    
    # TOmar la informacion por sujeto
    subj_obs = obs_df[obs_df["subject"] == subject]
    
    # Cada segmento determinar la el tiempo y asignar el miedo
    for seg_idx, seg in enumerate(segments):
        start_time = seg_idx * step_size  # aproximación
        center_time = start_time + window_size // 2
        
        # Buscar el evento mas cercano al tiempo:
        if not subj_obs.empty:
            subj_obs = subj_obs.copy()
            subj_obs.loc[:, "diff"] = np.abs(subj_obs["time"] - center_time)
            closest_event = subj_obs.loc[subj_obs["diff"].idxmin()]
            label = closest_event["fear level"]
        else:
            # si no funciona mejor saltarselo
            continue
        
        dataset_segments.append(seg)
        dataset_labels.append(label)

# Convertir a arrays
X_segments = np.array(dataset_segments)
y_segments = np.array(dataset_labels)

print(X_segments)
print(y_segments)

print("Número de muestras segmentadas:", X_segments.shape, y_segments.shape)

[[611. 611. 593. ... 618. 583. 606.]
 [665. 688. 618. ... 559. 657. 663.]
 [614. 572. 559. ... 552. 580. 621.]
 ...
 [767. 746. 747. ... 993. 987. 973.]
 [959. 980. 993. ... 922. 910. 926.]
 [929. 911. 922. ... 818. 868. 820.]]
[0 0 0 ... 0 0 0]
Número de muestras segmentadas: (1336, 10) (1336,)


In [7]:
from sklearn.neighbors import KNeighborsClassifier

# Definir la función de distancia DTW
def dtw_distance(x, y):
    # Aplanar los arrays porque si no no funciona
    x = np.asarray(x).ravel()
    y = np.asarray(y).ravel()
    
    # Verificar que x e y sean arrays 1-D
    if x.ndim != 1 or y.ndim != 1:
        raise ValueError(f"Forma de x: {x.shape}, Forma de y: {y.shape}")
    
    # Implementación personalizada de la distancia euclidiana
    def custom_euclidean(u, v):
        u = np.asarray(u).ravel()
        v = np.asarray(v).ravel()
        return np.linalg.norm(u - v)
    
    # Calcular la distancia DTW
    distance, _ = fastdtw(x, y, dist=custom_euclidean)
    
    return distance

In [8]:
knn = KNeighborsClassifier(n_neighbors=3, metric=dtw_distance)

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_segments, y_segments, test_size=0.3, random_state=42)

X_train = np.array([seg.ravel() for seg in X_train])
X_test = np.array([seg.ravel() for seg in X_test])

# Entrenar el modelo
knn.fit(X_train, y_train)

# Predecir
y_pred = knn.predict(X_test)

# Evaluar el modelo
print("Accuracy con DTW kNN:", accuracy_score(y_test, y_pred))
print("Reporte de clasificación:")
print(classification_report(y_test, y_pred))

Accuracy con DTW kNN: 0.6408977556109726
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.75      0.85      0.80       281
           1       0.07      0.06      0.06        18
           2       0.25      0.10      0.15        29
           3       0.00      0.00      0.00        11
           4       0.17      0.05      0.08        37
           5       0.30      0.44      0.35        25

    accuracy                           0.64       401
   macro avg       0.26      0.25      0.24       401
weighted avg       0.58      0.64      0.60       401



In [11]:
# Escalado de variables
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Aplicar PCA
pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

#Entrenamiento
knn.fit(X_train_pca, y_train)

# Predecir con una barra de progreso
y_pred = knn.predict(X_test_pca)
# Evaluar el modelo
print("Accuracy con DTW kNN y PCA:", accuracy_score(y_test, y_pred))
print("Reporte de clasificación:")
print(classification_report(y_test, y_pred))

Accuracy con DTW kNN y PCA: 0.6408977556109726
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.73      0.87      0.79       281
           1       0.06      0.06      0.06        18
           2       0.18      0.07      0.10        29
           3       0.00      0.00      0.00        11
           4       0.22      0.05      0.09        37
           5       0.32      0.28      0.30        25

    accuracy                           0.64       401
   macro avg       0.25      0.22      0.22       401
weighted avg       0.57      0.64      0.59       401



Finalmente mis dos modelos aun si intento escalar los datos parece no hacer nada, presumiblemente porque solamente estoy tomando una serie de datos y no multiples atributos. Otros intentos que hice fue utilizar otros modelos y utilizar los datos que la aplicacion de kubios genera pero los modelos con esos datos parecen ir peorprobablemente debido a que no logre acomodar bien los datos con las observaciones del miedo y los otros datos. Otros modelos que intente tuvieron accuracy de 0.33 y 0.51, esos utilizaren Randomforest y un intento de dtw pero no logre hacer que funcionara como esperaba. Finalmente este modelo con DTW parece ser el mejor con 0.64.